In [1]:
# Primero leo los CSV para poder armar los datasets
import pandas as pd

df_train = pd.read_csv("./fashion-mnist_train.csv")
df_test = pd.read_csv("./fashion-mnist_test.csv")

In [2]:
# Construyo mis datasets. En un principio tomo únicamente el csv de train provisto por Kaggle
# pero la idea es hacer k-fold cross-validation con lo cual luego voy a estar variando esto
x = df_train.drop(columns=["label"]).values
y = df_train["label"].values

#x_test = df_test.drop(columns=["label"]).values
#y_test = df_test["label"].values

In [3]:
# Transformo mi dataset con PCA para reducir la dimensión manteniendo la información más importante
from sklearn.decomposition import PCA
pca = PCA(n_components=50) # n_components es la cantidad de componentes que mantiene como mínimo la transformación. 
# Arranqué con una matriz de 60000x784 así que llegar a 60000x50 está bastante bien -> la idea es ir variando esto
# a ver cómo va variando el resultado
x_transformed = pca.fit_transform(x)
#x_test_transformed = pca.fit_transform(x_test)

In [4]:
# Transformo mi dataset con t-SNE para reducir la dimensión manteniendo la información más importante
from sklearn.manifold import TSNE
tSNE = TSNE(n_components=2)

x_tsne = tSNE.fit_transform(x_transformed)
x_tsne

array([[-14.642992 , -11.064627 ],
       [ 22.373596 , -41.45158  ],
       [-26.83103  ,  26.73778  ],
       ...,
       [  1.2708973,  51.86006  ],
       [-19.136173 ,  49.779526 ],
       [ 44.825134 , -34.239735 ]], dtype=float32)

In [ ]:
# Habiendo hecho la tranformación, armo el modelo
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

# Quiero hacer k-fold cross validation con mi set de train. Para eso, primero armo los folds y voy aplicando knn
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(x_tsne):
    x_train, x_test = x_transformed[train_index], x_transformed[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(x_train, y_train)
    knn.predict(x_test)
    
    print(knn.score(x_test, y_test))

# n_neighbors es el número de vecinos entre los cuales debe haber 
# una mayoría para setear el label correspondiente 
# (ej. si 3 de los 5 vecinos tienen label X entonces el nuevo punto es X)
#knn = KNeighborsClassifier(n_neighbors=10)
#knn.fit(x_train_transformed, y_train)
#knn.predict(x_test_transformed)
#knn.score(x_test_transformed, y_test)

0.85925
0.8563333333333333
0.861
